In [184]:
import pandas as pd

from PIL import Image
import os
import re
import shutil
from sklearn.model_selection import train_test_split
import sklearn
from sklearn import preprocessing

In [2]:
url=(r'C:\Users\Wdtum\Downloads\ofid_labels_v011\ofid_labels_bbox_v011.csv')
df=pd.read_csv(url)

In [3]:
#кодируем столбец метками 
le = preprocessing.LabelEncoder()

In [4]:
df['label_l1' ] = le.fit_transform(df['label_l1'])
print(le.classes_)

['Albacore' 'Bigeye tuna' 'Black marlin' 'Blue marlin' 'Great barracuda'
 'Human' 'Indo Pacific sailfish' 'Lancetfish' 'Long snouted lancetfish'
 'Mahi mahi' 'Marlin' 'No fish' 'Oilfish' 'Opah' 'Pelagic stingray'
 'Rainbow runner' 'Shark' 'Shortbill spearfish' 'Sickle pomfret'
 'Skipjack tuna' 'Striped marlin' 'Swordfish' 'Thresher shark'
 'Unknown fish' 'Wahoo' 'White margate' 'Yellowfin tuna']


In [5]:
df.head()

,img_id,bbox_id,x_min,x_max,y_min,y_max,label_l1,label_l2
0,94a69b66-23f0-11e9-a78e-2f2b7983ac0d,1,438,529,0,132,5,HUMAN
1,94a6a3a4-23f0-11e9-a78f-ebd9c88ef3e8,1,433,529,0,131,5,HUMAN
2,94a6a430-23f0-11e9-a790-2b5f72f1667a,1,440,529,0,132,5,HUMAN
3,94a6a48a-23f0-11e9-a791-fb958b6ab6b3,1,452,550,0,154,5,HUMAN
4,94a6a4da-23f0-11e9-a792-f320b734bd9b,1,462,550,0,153,5,HUMAN


In [38]:
import os
image_folder=r'C:\Users\Wdtum\Downloads\foid_images_v012\images'

In [39]:
#получаем список всех файлов
image_files=os.listdir(image_folder)
len(image_files)

34987

In [35]:
image[0:10]

['94a69b66-23f0-11e9-a78e-2f2b7983ac0d.jpg',
 '94a6a3a4-23f0-11e9-a78f-ebd9c88ef3e8.jpg',
 '94a6a430-23f0-11e9-a790-2b5f72f1667a.jpg',
 '94a6a48a-23f0-11e9-a791-fb958b6ab6b3.jpg',
 '94a6a4da-23f0-11e9-a792-f320b734bd9b.jpg',
 '94a6a534-23f0-11e9-a793-c7e8fecc9fa8.jpg',
 '94a6a584-23f0-11e9-a794-0fe64977c9b7.jpg',
 '94a6a5de-23f0-11e9-a795-6b168e42914e.jpg',
 '94a6a62e-23f0-11e9-a796-f783fa7ee4cf.jpg',
 '94a6a688-23f0-11e9-a797-9371b85a8f67.jpg']

In [45]:
#получаем информацию об изображениях
image_info=[]
for image_file in image_files:
    image_path=os.path.join(image_folder,image_file)
    with Image.open(image_path) as img:
        width,height=img.size
        image_info.append([image_file.replace('.jpg',''),width,height])

In [54]:
#создаем новый датасет с информацией об изображениях
columns=['img_id','width_img','height_img']
dataset_image=pd.DataFrame(image_info,columns=columns)

In [55]:
#сохраняем его
dataset_image.to_csv('info_image.csv',index=False)

In [56]:
dataset_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34987 entries, 0 to 34986
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   img_id      34987 non-null  object
 1   width_img   34987 non-null  int64 
 2   height_img  34987 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 820.1+ KB


In [58]:
#проверка размеров изображений(вдруг они были все одного размера)
print(dataset_image['width_img'].unique())
print(dataset_image['height_img'].unique())

[1280 1200 1208 1334 1192 1518 1276 1520 1224 1244 1156 1732]
[720 675 755 750 670 974 854 718 723 859 979 700 650 924]


In [61]:
#совмещаем два датасета
res_data=df.merge(dataset_image,on=['img_id'])
res_data.head()

,img_id,bbox_id,x_min,x_max,y_min,y_max,label_l1,label_l2,width_img,height_img
0,94a69b66-23f0-11e9-a78e-2f2b7983ac0d,1,438,529,0,132,5,HUMAN,1280,720
1,94a6a3a4-23f0-11e9-a78f-ebd9c88ef3e8,1,433,529,0,131,5,HUMAN,1280,720
2,94a6a430-23f0-11e9-a790-2b5f72f1667a,1,440,529,0,132,5,HUMAN,1280,720
3,94a6a48a-23f0-11e9-a791-fb958b6ab6b3,1,452,550,0,154,5,HUMAN,1280,720
4,94a6a4da-23f0-11e9-a792-f320b734bd9b,1,462,550,0,153,5,HUMAN,1280,720


In [62]:
res_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162804 entries, 0 to 162803
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   img_id      162804 non-null  object
 1   bbox_id     162804 non-null  int64 
 2   x_min       162804 non-null  int64 
 3   x_max       162804 non-null  int64 
 4   y_min       162804 non-null  int64 
 5   y_max       162804 non-null  int64 
 6   label_l1    162804 non-null  int32 
 7   label_l2    162804 non-null  object
 8   width_img   162804 non-null  int64 
 9   height_img  162804 non-null  int64 
dtypes: int32(1), int64(7), object(2)
memory usage: 13.0+ MB


In [149]:
#создание датасета для обучения модели
data=pd.DataFrame(columns=['img_id','class','x_center','y_center','width','height'])
data.head()

,img_id,class,x_center,y_center,width,height


In [150]:
#вычисляем данные о центрах рамки и ее габаритах
data['width']=res_data['x_max']-res_data['x_min']
data['height']=res_data['y_max']-res_data['y_min']
data['x_center']=res_data['x_min']+ data['width']/2
data['y_center']=res_data['y_min']+ data['height']/2
data['class']=res_data['label_l1']
data['img_id']=res_data['img_id']

In [151]:
data.head(10)

,img_id,class,x_center,y_center,width,height
0,94a69b66-23f0-11e9-a78e-2f2b7983ac0d,5,483.5,66.0,91,132
1,94a6a3a4-23f0-11e9-a78f-ebd9c88ef3e8,5,481.0,65.5,96,131
2,94a6a430-23f0-11e9-a790-2b5f72f1667a,5,484.5,66.0,89,132
3,94a6a48a-23f0-11e9-a791-fb958b6ab6b3,5,501.0,77.0,98,154
4,94a6a4da-23f0-11e9-a792-f320b734bd9b,5,506.0,76.5,88,153
5,94a6a534-23f0-11e9-a793-c7e8fecc9fa8,5,463.5,92.0,135,162
6,94a6a534-23f0-11e9-a793-c7e8fecc9fa8,5,225.0,392.0,222,132
7,94a6a584-23f0-11e9-a794-0fe64977c9b7,5,227.5,391.5,247,137
8,94a6a584-23f0-11e9-a794-0fe64977c9b7,5,462.0,91.0,138,160
9,94a6a5de-23f0-11e9-a795-6b168e42914e,5,464.0,92.5,136,161


In [152]:
#нормализуем данные
data['width']= data['width']/res_data['width_img']
data['height']= data['height']/res_data['height_img']
data['x_center']=data['x_center']/res_data['width_img'] 
data['y_center']=data['y_center']/res_data['height_img'] 
 

In [73]:
 data.head()

,img_id,class,x_center,y_center,width,height
0,94a69b66-23f0-11e9-a78e-2f2b7983ac0d,5,0.377734,0.091667,0.071094,0.183333
1,94a6a3a4-23f0-11e9-a78f-ebd9c88ef3e8,5,0.375781,0.090972,0.075000,0.181944
2,94a6a430-23f0-11e9-a790-2b5f72f1667a,5,0.378516,0.091667,0.069531,0.183333
3,94a6a48a-23f0-11e9-a791-fb958b6ab6b3,5,0.391406,0.106944,0.076563,0.213889
4,94a6a4da-23f0-11e9-a792-f320b734bd9b,5,0.395313,0.106250,0.068750,0.212500


In [176]:
#создаем для каждого изображения одноименный файл с информацией об объектах, которые там находятся
direct=r'C:\Users\Wdtum\OneDrive\Рабочий стол\lables'
REPLACE_BY_SPACE_RE=re.compile('[/(){}\[\]\|@,;]')
for image_file in image_files:
    image_file=image_file.replace('.jpg','')
    file=open(os.path.join(direct,f'{image_file}.txt'),'w')
    rem=str(data.loc[data['img_id'] == image_file.replace('.jpg',''),['class','x_center','y_center','width','height']].values)
    rem=re.sub(image_file.replace('.jpg',''),'',rem)
    REPLACE_BY_SPACE_RE.sub(' ',rem)
    file.write(REPLACE_BY_SPACE_RE.sub(' ',rem))
    file.close()

In [177]:
#проверям размерность
lables=os.listdir(direct)
len(lables)

34987

In [179]:
#создаем папки для деления данных на тестовую и тренировочную выборку
# Путь к папке с данными
data_folder = r"C:\Users\Wdtum\Downloads\foid_images_v012\images"""

# Путь к папке с метками
labels_folder = r"C:\Users\Wdtum\OneDrive\Рабочий стол\lables"

# Путь к папке тренировочных данных
train_data_folder = r"C:\Users\Wdtum\OneDrive\Рабочий стол\data_yolo\train_data_folder"

# Путь к папке тренировочных меток
train_labels_folder = r"C:\Users\Wdtum\OneDrive\Рабочий стол\data_yolo\train_labels_folder"

# Путь к папке тестовых данных
test_data_folder = r"C:\Users\Wdtum\OneDrive\Рабочий стол\data_yolo\test_data_folder"

# Путь к папке тестовых меток
test_labels_folder = r"C:\Users\Wdtum\OneDrive\Рабочий стол\data_yolo\test_labels_folder"

In [181]:
#проверка созданы папки или нет
os.makedirs(train_data_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(test_data_folder, exist_ok=True)
os.makedirs(test_labels_folder, exist_ok=True)

In [182]:
# Получение списка имен файлов из папки с данными
data_files = os.listdir(data_folder)

# Получение списка имен файлов из папки с метками
labels_files = os.listdir(labels_folder)

In [185]:
# Разделение файлов на тренировочные и тестовые выборки
train_data, test_data, train_labels, test_labels = train_test_split(
    data_files, labels_files, test_size=0.2, random_state=42
)

In [186]:
# Копирование тренировочных данных и меток в соответствующие папки
for filename in train_data:
    shutil.copy(os.path.join(data_folder, filename), os.path.join(train_data_folder, filename))
for filename in train_labels:
    shutil.copy(os.path.join(labels_folder, filename), os.path.join(train_labels_folder, filename))

 

In [187]:
# Копирование тестовых данных и меток в соответствующие папки
for filename in test_data:
    shutil.copy(os.path.join(data_folder, filename), os.path.join(test_data_folder, filename))
for filename in test_labels:
    shutil.copy(os.path.join(labels_folder, filename), os.path.join(test_labels_folder, filename))

In [188]:
%pip install PyYAML==5.1

     -------------------------------------- 274.2/274.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for PyYAML: filename=PyYAML-5.1-cp39-cp39-win_amd64.whl size=44085 sha256=1a3550313be87c22ebf3cadb6c9c3b4ceb617dafed66f78be5e9e1e6b20c83ad
  Stored in directory: c:\users\wdtum\appdata\local\pip\cache\wheels\68\be\8f\b6c454cd264e0b349b47f8ee00755511f277618af9e5dae20d
Successfully built PyYAML
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.0.171 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
pandas-profiling 3.5.0 requires numpy<1.24,>=1.16.0, but you have numpy 1.24.4 which is incompatible.
dask 2022.7.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.20 requires PyYAML==6.0, but you have pyyaml 5.1 which is incompatible.


In [189]:
import yaml

In [193]:
#создаем yaml файл для подачи его потом в нейронную сеть
data = [{
    'train': r'C:\Users\Wdtum\OneDrive\Рабочий стол\data_yolo\train\images',
    'val': r'C:\Users\Wdtum\OneDrive\Рабочий стол\data_yolo\val\images',
    'nc': 27,
    'names': ['Albacore','Bigeye tuna', 'Black marlin', 'Blue marlin', 'Great barracuda',
 'Human', 'Indo Pacific sailfish', 'Lancetfish','Long snouted lancetfish',
 'Mahi mahi' ,'Marlin' ,'No fish' ,'Oilfish', 'Opah', 'Pelagic stingray',
 'Rainbow runner', 'Shark', 'Shortbill spearfish', 'Sickle pomfret',
 'Skipjack tuna', 'Striped marlin', 'Swordfish', 'Thresher shark',
 'Unknown fish', 'Wahoo', 'White margate', 'Yellowfin tuna']
}]

In [194]:
def write_yaml_to_file(py_obj, filename) :
    with open(f'{filename}.yaml', 'w+',) as f:
        yaml.dump_all(py_obj, f, sort_keys=False)
write_yaml_to_file(data, 'yolo')